In [ ]:
import pandas as pd
from datetime import datetime, timedelta

base_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"

# Generar fechas de febrero 2020
dates = pd.date_range(start='2020-02-01', end='2020-02-29', freq='D')

dfs = []
for date in dates:
    try:
        # Formato: MM-DD-YYYY
        filename = date.strftime('%m-%d-%Y') + '.csv'
        url = base_url + filename
        print(f"Cargando: {filename}")
        
        df = pd.read_csv(url)
        df['Date'] = date  # Agregar columna de fecha
        dfs.append(df)
    except Exception as e:
        print(f"Error con {filename}: {e}")

# Concatenar todos los DataFrames
df_enero = pd.concat(dfs, ignore_index=True)
print(f"\nTotal de registros: {len(df_enero)}")
print(f"Fechas cargadas: {df_enero['Date'].min()} a {df_enero['Date'].max()}")

: 